In [12]:
import pandas as pd
import missingno as msno
from data_preprocessing import preprocessing

data = pd.read_csv("../../data/Ana Health_Tabela Modelo Previsão Churn - Tabela.csv")

In [13]:
len(data)

1203

In [14]:
data = preprocessing(data)

NameError: name 'SimpleImputer' is not defined

In [15]:
len(data)

8

In [16]:
data.to_csv("data-preprocessed.csv", index=False)